# Recommendation system using TensorFlow

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load the data
df = pd.read_csv('kurtiData.csv')

C:\Users\EEE\anaconda3\envs\pytorchenv\lib\site-packages\sklearn\utils\validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


In [2]:
print(df.head())

                            user_id product_id  \
0  0b32cb93b4d1467a895fdd0c0fdcac7d     1xiamj   
1  a546c81dcb9346619c00f3632bb39318     1xiamj   
2  6f7480ff712d42eb9be53f37641a3bf9     1xiamj   
3  cf05abc75bd74b159bc91a76ad4bfbb5     1xiamj   
4  893a87b0ef41435cbd40f5065bdff448     1xiamj   

                                         product_url  \
0  https://www.meesho.com/alisha-refined-kurtis/p...   
1  https://www.meesho.com/alisha-refined-kurtis/p...   
2  https://www.meesho.com/alisha-refined-kurtis/p...   
3  https://www.meesho.com/alisha-refined-kurtis/p...   
4  https://www.meesho.com/alisha-refined-kurtis/p...   

                                           image_url price  rating   timestamp  
0  https://images.meesho.com/images/products/1167...  ₹293       5  1816526580  
1  https://images.meesho.com/images/products/1167...  ₹293       5  1431790209  
2  https://images.meesho.com/images/products/1167...  ₹293       5  1455698017  
3  https://images.meesho.com/images/pr

In [4]:
# Select relevant columns for the recommendation system
df = df[['user_id', 'product_id', 'rating']]

In [5]:
# Encode user_id and product_id as categorical data
df['user_id'] = df['user_id'].astype('category').cat.codes
df['product_id'] = df['product_id'].astype('category').cat.codes

In [6]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

## Build the Model

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense

# Define the number of unique users and products
num_users = df['user_id'].nunique()
num_products = df['product_id'].nunique()

# Define the embedding dimension
embedding_dim = 50

# Define the model
class RecommenderModel(Model):
    def __init__(self, num_users, num_products, embedding_dim):
        super(RecommenderModel, self).__init__()
        self.user_embedding = Embedding(num_users, embedding_dim, embeddings_initializer='he_normal', embeddings_regularizer=tf.keras.regularizers.l2(1e-6))
        self.product_embedding = Embedding(num_products, embedding_dim, embeddings_initializer='he_normal', embeddings_regularizer=tf.keras.regularizers.l2(1e-6))
        self.dot = Dot(axes=1)
    
    def call(self, inputs):
        user_vector = self.user_embedding(inputs[:, 0])
        product_vector = self.product_embedding(inputs[:, 1])
        dot_product = self.dot([user_vector, product_vector])
        return dot_product

# Instantiate the model
model = RecommenderModel(num_users, num_products, embedding_dim)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')

# Prepare the training data
train_inputs = train_data[['user_id', 'product_id']].values
train_targets = train_data['rating'].values

# Train the model
model.fit(train_inputs, train_targets, epochs=5, batch_size=64, verbose=1)


Epoch 1/5
5325/5325 [==============================] - 138s 26ms/step - loss: 7.9666
Epoch 2/5
5325/5325 [==============================] - 131s 25ms/step - loss: 2.6386
Epoch 3/5
5325/5325 [==============================] - 136s 25ms/step - loss: 2.4224
Epoch 4/5
5325/5325 [==============================] - 133s 25ms/step - loss: 2.2498
Epoch 5/5
5325/5325 [==============================] - 131s 25ms/step - loss: 1.9740


## A/B testing framework

In [12]:
# Define the A/B testing framework
import random
class ABTest:
    def __init__(self, model_a, model_b):
        self.model_a = model_a
        self.model_b = model_b
        self.results = {'A': [], 'B': []}

    def assign_group(self, user_id):
        return 'A' if random.random() < 0.5 else 'B'

    def recommend_products(self, user_id, group, num_recommendations=5):
        user_vector = np.array([user_id] * num_products)
        product_vector = np.array(range(num_products))
        inputs = np.column_stack((user_vector, product_vector))
        
        if group == 'A':
            predictions = self.model_a.predict(inputs)
        else:
            predictions = self.model_b.predict(inputs)
        
        top_product_ids = np.argsort(predictions.flatten())[-num_recommendations:]
        return top_product_ids

    def track_interaction(self, group, interaction):
        self.results[group].append(interaction)

    def analyze_results(self):
        avg_interaction_a = np.mean(self.results['A'])
        avg_interaction_b = np.mean(self.results['B'])
        print(f"Average interaction for Group A: {avg_interaction_a}")
        print(f"Average interaction for Group B: {avg_interaction_b}")

# Instantiate the A/B testing framework
ab_test_framework = ABTest(model, model)

# Example usage
user_id = 0  # Example user_id
group = ab_test_framework.assign_group(user_id)
recommended_products = ab_test_framework.recommend_products(user_id, group)
print(f"Group: {group}, Recommended products: {recommended_products}")

# Simulate tracking user interaction
interaction = random.choice([0, 1])  # Example interaction: 0 for no interaction, 1 for interaction
ab_test_framework.track_interaction(group, interaction)

# Analyze results
ab_test_framework.analyze_results()

30/30 [==============================] - 0s 560us/step
Group: A, Recommended products: [619 508 899 205 676]
Average interaction for Group A: 0.0
Average interaction for Group B: nan
